<a href="https://colab.research.google.com/github/magnolia2001/Forest_Estimation/blob/main/notebooks/NNandCNN_FeatureNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 在划分训练集和测试集之前对特征进行标准化

且不进行数据平衡

In [ ]:
from google.colab import drive

# 挂载 Google Drive
drive.mount('/content/drive')

# 检查挂载的路径结构
!ls /content/drive

In [ ]:
root_path = '/content/drive/My Drive/data/'
path_images = f'{root_path}images/'
path_masks = f'{root_path}masks/'

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime, os, cv2
from matplotlib import pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, mean_absolute_percentage_error as mape
# from keras.models import Sequential, load_model
# from keras.layers import Dense, BatchNormalization, Dropout, InputLayer, Flatten, Conv2D, MaxPool2D, AveragePooling2D
# from keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, InputLayer, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint


# 制作标签数据和特征数据

# 窗口大小应为奇数，以保证标签在中间
size = 5 #define window size should be odd so that the label is in the middle
# 特征的形状，这里假设每个特征是一个大小为 (size, size) 的窗口，包含 11 个通道
shape = (11, size, size) #define shape of features
# np.ones(shape, dtype=None) 用于创建一个形状为 shape 的数组，并将所有元素初始化为 1.0
# 其中 shape：指定数组的形状，通常是一个整数或元组。 dtype：指定数组元素的数据类型（可选）。如果不指定，默认使用 float64 类型。
# labels1 用于存放标签数据（掩膜）, data1 用于存放提取的特征数据
# np.ones(1)返回的是一个只有一个元素的数组，其中该元素值为 1。
labels1 = np.ones(1) #array for labels
# 创建了一个数组，形状为 shape 即 (11, 5, 5) 的 NumPy 数组，并且所有的元素值都被初始化为 1.0 。
data1 = np.ones(shape) #array for features
# 扩展维度，便于后续拼接操作
data1 = np.expand_dims(data1, axis=0) #expand dimension to concatenate

# 遍历目录中的图像（假设有 20 张图像, 具体数量还需要根据自己的情况修改）
# 在 for j in range(20) 这个遍历过程中，区分 j < 10 和 j >= 10 的目的是为了处理不同的文件命名规则。
# 对于小于 10 的文件名，文件名是 "image_00X.npy"，其中 X 是单个数字（0 到 9）。
# 对于大于等于 10 的文件名，文件名是 "image_0XY.npy"，其中 XY 是两位数的数字（10 到 19）。
for j in range(142): #iterate over images in directory
  if j < 10:
    # 路径填写实际路径
    # 读取图像数据
    X = np.load(f'{path_images}image_00'+ str(j) + '.npy')
    # 读取掩膜数据
    y = np.load(f'{path_masks}mask_00'+ str(j) + '.npy')
    # 移除掩膜图像中的通道维度使其形状变为(height, width)
    # y = y[0, :, :]  # 去掉通道维度，保留二维掩膜图像

    # 选择掩膜图像 y 中所有大于 0 的位置（即标签不为 0 的位置），并返回这些位置的索引。
    # indices 数组返回 N 个元素，其中 N 为掩膜图像 y 中所有大于 0 的元素个数。每个元素都是一个长度为 2 的行向量，表示符合条件元素的行列索引。
    # y > 0 是一个布尔条件，返回一个与 y 相同形状的布尔数组, 如果是大于 0，布尔值为 True，否则为 False。
    # np.argwhere() 是 NumPy 库中的一个函数，它返回数组中满足某个条件的所有索引（行列坐标），即满足条件的元素的坐标位置。
    # 这里 indices 是一个形状为 (N, 2) 的二维数组，每一行是 (y, x) 坐标. y 为行索引, x 为列索引
    indices = np.argwhere(y > 0) #select all values with label

    # indices_2d 是 indices 数组的一个切片，是一个 二维数组, 它包含了所有掩膜图像中标签值大于 0 的位置的 列索引。
    # 切片操作 indices[:, 1:] 就是提取所有行中的第二列（即 行 和 列 坐标中的 列索引）。
    # indices_2d = indices[:, 1:] #extract indices

    # 初始化一个数组 ind_y 来收集符合条件的标签位置。
    # np.ones(2) 会创建一个包含 2 个元素的数组，所有元素的值为 1. 。
    # .reshape(-1, 2) 将该数组的形状重塑为 (-1, 2)，表示按列数为 2 进行重塑，-1 表示自动计算行数。由于只有 2 个元素，这会将数组变成形状为 (1, 2) 的二维数组。
    ind_y = np.ones(2).reshape(-1,2) #array to collect indices

    # 遍历掩膜中的每个标签位置
    # for i in indices_2d: #iterate over indices
    for i in indices:
      # 提取图像块，并检查其形状。
      # size//2 表示 size 除以 2 的整数部分，用于确定图像块中心点到边界的距离。右端点的值之所以加 1 是因为区间是左闭右开的,所以加 1 保证能取到右端点.
      # 整个i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2)表达式计算出一个范围，用于选取以 (i[0], i[1])（标签的 y, x 坐标） 为中心，上下各延伸 size//2 个像素的区域。
      # i[0] 是当前标签位置的 y 坐标（行索引）。i[1] 是当前标签位置的 x 坐标（列索引）。
      # 利用 shape 确保当前窗口大小与指定的窗口大小一致
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape: #select only features with the same shape because of labels at the image border
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1] #save them temporary
        # 将 temp 的维度扩展一个维度，使得它变成一个形状为 (1, channels, size, size) 的四维数组。扩展维度的目的是为了能够将 temp 与其他提取的窗口进行拼接。
        temp2 = np.expand_dims(temp, axis=0) #expand dimension to concatenate
        # 拼接特征数据
        # data1 最终会变成 (num_samples, 11, 5, 5)，其中 num_samples 是提取的窗口数量。
        data1 = np.concatenate((data1, temp2), axis=0) #concatenation
        # 拼接标签索引
        # i.reshape(-1, 2) 会把 i 重新调整为一个形状为 (1, 2) 的二维数组
        # axis=0 表示在 第 0 维（行方向） 进行拼接，即新添加的行会被添加到原数组的最后。
        # ind_y 则是一个 (num_samples, 2) 的数组，每个样本对应一个标签位置的 (y, x) 坐标。
        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0) #concatenation of index so that they have the same order and length as the features

    # 去掉第一个虚拟值
    # 初始时，ind_y 中的第一个元素是 np.ones(2).reshape(-1,2) 创建的虚拟数据。此步骤是将它移除，只保留实际的标签坐标。
    ind_y = ind_y[1:] #remove first dummy values
    # 提取所有的行索引
    indices_1 = ind_y[:, 0].astype(int)
    # 提取所有的列索引
    indices_2 = ind_y[:, 1].astype(int)
    # 提取标签值
    # 从这句代码应该可以看出原作者的掩膜图像的形状包含了通道维度,即形状为(1, height, width)
    # data_y = y[0, indices_1, indices_2] #extract labels
    data_y = y[indices_1, indices_2]  # 提取标签
    # 拼接标签，形成最终的标签数组。
    labels1 = np.concatenate((labels1, data_y), axis = 0) #concatenate labels

  if j >= 10 and j < 100:
    X = np.load(f'{path_images}image_0'+ str(j) + '.npy')
    y = np.load(f'{path_masks}mask_0'+ str(j) + '.npy')
    # 移除掩膜图像中的通道维度使其形状变为(height, width)
    # y = y[0, :, :]  # 去掉通道维度，保留二维掩膜图像
    indices = np.argwhere(y > 0)
    # indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    # for i in indices_2d:
    for i in indices:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data1 = np.concatenate((data1, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    # 去掉第一个虚拟值
    ind_y = ind_y[1:]
    # 提取所有的行索引
    indices_1 = ind_y[:, 0].astype(int)
    # 提取所有的列索引
    indices_2 = ind_y[:, 1].astype(int)
    # data_y = y[0, indices_1, indices_2]
    # 提取标签值
    # 每一对 (indices_1[i], indices_2[i]) 会自动匹配，得到对应位置的标签值。
    data_y = y[indices_1, indices_2]  # 提取标签
    # 拼接标签，形成最终的标签数组。
    labels1 = np.concatenate((labels1, data_y), axis = 0)

  if j >= 100:
    X = np.load(f'{path_images}image_'+ str(j) + '.npy')
    y = np.load(f'{path_masks}mask_'+ str(j) + '.npy')
    # 移除掩膜图像中的通道维度使其形状变为(height, width)
    # y = y[0, :, :]  # 去掉通道维度，保留二维掩膜图像
    indices = np.argwhere(y > 0)
    # indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    # for i in indices_2d:
    for i in indices:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data1 = np.concatenate((data1, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    # 去掉第一个虚拟值
    ind_y = ind_y[1:]
    # 提取所有的行索引
    indices_1 = ind_y[:, 0].astype(int)
    # 提取所有的列索引
    indices_2 = ind_y[:, 1].astype(int)
    # data_y = y[0, indices_1, indices_2]
    # 提取标签值
    # 每一对 (indices_1[i], indices_2[i]) 会自动匹配，得到对应位置的标签值。
    data_y = y[indices_1, indices_2]  # 提取标签
    # 拼接标签，形成最终的标签数组。
    labels1 = np.concatenate((labels1, data_y), axis = 0)

# 移除第一个虚拟值
# data1 的形状会是 (num_samples, 11, 5, 5)，其中 num_samples 是提取的窗口数量（即符合条件的标签数量）。一个四维数组
# labels1 的形状会是 (num_samples,)，其中 num_samples 是所有图像中符合条件的标签数量。一个一维数组
data1 = data1[1:] #remove first dummy values
labels1 = labels1[1:] #remove first dummy values

# data1 和 labels1 应该是 一一对应的，因为它们的样本数（num_samples）相同。
# 获得标签数据和特征数据
features = data1
labels = labels1


# Neural Network

In [ ]:

# features 数组形状为 (num_samples, 11, 5, 5)，意味着每个样本有 11 个特征（或 11 个通道），每个特征是一个 5x5 的空间窗口。
features_mean = np.mean(features, axis=(2, 3)) # patch mean of size * size features

# 现在 features_mean 的形状是 (num_samples, 11)，适用于NN或者其他传统机器学习算法
# 注意: train_test_split 只能处理 NumPy 数组或 Pandas DataFrame，并不能直接处理 TensorFlow Dataset 对象。因此，这部分代码在处理 TensorFlow Dataset 时会出错。
X_train, X_test, y_train, y_test = train_test_split(features_mean, labels, test_size = 0.3, random_state=3)


可以根据统计信息来决定是否需要对特征进行标准化处理

In [ ]:
# 打印 features_mean 的统计信息
print("features_mean 的统计信息：")
print(f"最大值: {np.max(features_mean)}")
print(f"最小值: {np.min(features_mean)}")
print(f"均值: {np.mean(features_mean)}")
print(f"标准差: {np.std(features_mean)}")


In [ ]:
# 针对每个特征维度（即 11 个通道）计算统计信息
for i in range(features_mean.shape[1]):  # 遍历 11 个特征通道
    print(f"特征通道 {i+1} 的统计信息：")
    print(f"  最大值: {np.max(features_mean[:, i])}")
    print(f"  最小值: {np.min(features_mean[:, i])}")
    print(f"  均值: {np.mean(features_mean[:, i])}")
    print(f"  标准差: {np.std(features_mean[:, i])}")


在标准化特征时，使用 训练集 来拟合标准化器（如 StandardScaler），并用其参数（均值和标准差）对测试集进行变换，这是一种 防止数据泄露 的必要操作。

标准化的正确流程：
1. 用训练集计算标准化参数（如均值和标准差）。
2. 用这些参数对训练集和测试集分别进行标准化。

In [ ]:
from sklearn.preprocessing import StandardScaler

# 初始化标准化器
scaler = StandardScaler()

# fit_transform()：计算训练集的统计参数（均值和标准差）。根据这些参数对数据进行标准化。适用于训练集
# 仅用训练集数据拟合标准化器（避免数据泄露）
X_train = scaler.fit_transform(X_train)

# transform()：不会重新计算统计参数，而是使用 fit_transform() 计算出的参数对数据进行标准化。适用于测试集（或者未来的预测数据）。
# 使用训练集的标准化参数变换测试集
X_test = scaler.transform(X_test)

In [ ]:
%load_ext tensorboard

### 神经网络模型（NN模型）定义

In [ ]:
# 使用 Sequential 模型，适用于线性堆叠的神经网络。
modelNn = Sequential()  # build neural network

# 第一层：Dense(128, input_shape=(11,), ...)代表输入层，输入形状为 11,)，表示每个输入样本有 11 个特征。这个 11 是硬编码的，但实际上它应该等于特征的维度数，这里可能需要根据实际情况进行修改。
# 如果特征数是动态变化的，建议不要硬编码这个数字，而是通过 features.shape[1] 获取动态的特征维度。

input_dim = features.shape[1]  # 动态获取特征维度
modelNn.add(Dense(128, input_shape=(input_dim,), kernel_initializer='normal', activation='relu'))

# 第二层和第三层：分别包含 256 个神经元，也是全连接层，使用 ReLU 激活函数。
modelNn.add(Dense(256, kernel_initializer='normal', activation='relu'))
modelNn.add(Dense(256, kernel_initializer='normal', activation='relu'))
modelNn.add(Dense(128, kernel_initializer='normal', activation='relu'))

# 使用 Dropout(0.4) 防止过拟合，随机丢弃 40% 的神经元
modelNn.add(Dropout(0.4))

# 输出层包含 1 个神经元，activation='linear' 表示线性激活函数，适用于回归任务。
modelNn.add(Dense(1, kernel_initializer='normal', activation='linear'))


# 打印模型的结构，包括每一层的类型、输出形状、参数数量等信息，帮助你了解模型的结构。
modelNn.summary()


In [ ]:
# 损失函数：loss='mean_absolute_error' 使用绝对误差作为回归问题的损失函数。
# 优化器：optimizer='adam' 使用 Adam 优化器，它是目前常用的一种高效的优化算法。
# 评估指标：metrics=['mean_absolute_percentage_error'] 使用平均绝对百分比误差（MAPE）作为评估指标。

# 注意: 在训练过程中，优化算法会根据 val_loss 来更新模型的参数，因为 val_loss 是损失函数的值，而损失函数通常是模型优化的目标。
#    MAPE 作为评估指标，不会直接影响模型的参数更新，它仅用于评价模型在验证集上的相对误差，帮助你了解模型的实际表现。
modelNn.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error']) #compile model


In [ ]:
# 生成一个包含当前时间戳的日志目录路径。为 TensorBoard 准备日志文件存储位置。
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))  # log directory for tensorboard

# 在训练过程中将日志信息保存到指定的 logdir 目录，并设置每个 epoch 保存权重的直方图。在训练过程中启用 TensorBoard 回调，确保训练日志被记录。
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

# 模型保存, 仅保存表现最好的模型。
# HDF5 格式的模型文件，请确保扩展名为 .h5 或 .hdf5
os.makedirs("/content/drive/My Drive/NNmodels", exist_ok=True)

model_save = ModelCheckpoint(
    "/content/drive/My Drive/NNmodels/best_NNmodel.keras",
    save_best_only=True,
    save_weights_only=False  # 保存完整模型（包括架构、权重和优化器状态）
)


In [ ]:
# 使用 model.fit() 训练模型：
modelNn.fit(X_train, y_train, epochs = 100, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, model_save]) #fit model

In [ ]:
# Keras 中的函数，用于加载深度学习模型。
bmodel = load_model('/content/drive/My Drive/NNmodels/best_NNmodel.keras') #load best model

In [ ]:
# 使用训练好的模型对测试集 X_test 进行预测，返回预测值 ypred_nn。
ypred_nn = bmodel.predict(X_test)


In [ ]:
# 计算测试集的均方误差。 (MSE)
mse_nn = mse(y_test, ypred_nn)
# 计算均方根误差 (RMSE)
rmse_nn = mse_nn ** (1/2)
# 计算平均绝对误差 (MAE)
mae_nn = mae(y_test, ypred_nn)
# 平均绝对百分比误差 (MAPE)
mape_nn = mape(y_test, ypred_nn)

# 打印出 MAPE、MAE 和 RMSE 评估指标，帮助你评估模型的表现
print(mape_nn)
print(mae_nn)
print(rmse_nn)


In [ ]:
%tensorboard --logdir logs

In [ ]:
# 计算预测结果的均值
mean_nn = np.mean(ypred_nn[:])  # calculate mean
# 计算预测结果的不同分位数（1%, 25%, 50%, 75%, 99%）
quantiles_nn = np.percentile(ypred_nn[:], [1, 25, 50, 75, 99])  # calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99

# 计算标签的均值
mean_labels = np.mean(labels[:])
# 计算标签的不同分位数（1%, 25%, 50%, 75%, 99%）
quantiles_labels = np.percentile(labels[:], [1, 25, 50, 75, 99])

print(mean_nn)
print(quantiles_nn)
# 打印预测值中最小的 10 个和最大的 10 个。
print(np.sort(ypred_nn.flatten())[:10])  # print the 10 lowest predictions
print(np.sort(ypred_nn.flatten())[-10:][::-1])  # print the 10 highest predictions

print(mean_labels)
print(quantiles_labels)
# 打印标签中最小的 10 个和最大的 10 个。
print(np.sort(labels.flatten())[:10])
print(np.sort(labels.flatten())[-10:][::-1])


# Convolutional neural network

In [ ]:
# 数据集拆分：训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.3, random_state=3) #create train, test set


In [ ]:
from sklearn.preprocessing import StandardScaler

# 初始化标准化器
scaler = StandardScaler()

# fit_transform()：计算训练集的统计参数（均值和标准差）。根据这些参数对数据进行标准化。适用于训练集
# 仅用训练集数据拟合标准化器（避免数据泄露）
X_train = scaler.fit_transform(X_train)

# transform()：不会重新计算统计参数，而是使用 fit_transform() 计算出的参数对数据进行标准化。适用于测试集（或者未来的预测数据）。
# 使用训练集的标准化参数变换测试集
X_test = scaler.transform(X_test)

In [ ]:
# 加载 TensorBoard 插件，用于可视化训练过程的日志信息，如损失曲线、指标等。
%load_ext tensorboard

In [ ]:
# 转换 X_train 和 X_test 的格式为 NHWC
# y_train 和 y_test 是目标标签，它们通常是数值（对于回归问题）或分类标签（对于分类问题），是一维数组。目标标签与 CNN 的数据格式无关，因此无需调整。
X_train = X_train.transpose(0, 2, 3, 1)  # 从 (N, C, H, W) 转换为 (N, H, W, C)
X_test = X_test.transpose(0, 2, 3, 1)

# 检查转换后的形状
print(X_train.shape)  # 应输出 (49991, 5, 5, 11)
print(X_test.shape)   # 应输出 (21426, 5, 5, 11)


In [ ]:
# Keras 默认使用 channels_last 格式，这表示图像的输入形状是 (height, width, channels)。如果想使用 channels_first 格式（即 (channels, height, width)），需要进行相应的设置，比如通过 Keras 配置全局设置或者在模型层中明确指定
# 这里采用为每个层指定数据格式。只有卷积层和池化层需要明确指定数据格式（channels_first 或 channels_last），而全连接层不需要这样做，展平层会处理掉格式问题。
# 卷积操作会根据指定的数据格式进行计算，输出的形状也会遵循这种数据格式。

# 使用 Sequential() 创建一个顺序模型（即按顺序堆叠各层）。
modelCnn = Sequential() #bulid cnn

# 因为 features 数组形状为 (num_samples, 5, 5, 11)
# 指定输入数据的形状。这里假设每个输入图像是一个 11 个通道的 5x5 大小的图像块。
modelCnn.add(InputLayer(input_shape=(5, 5, 11)))  # NHWC 格式 # 输入形状为 (height, width，channels)

# 注意：在卷积神经网络中，卷积核的形状是 (高度, 宽度, 输入通道数, 输出通道数)，这代表每个卷积核的尺寸、它需要处理的输入通道数，以及它生成的输出通道数。
# 高度和宽度决定了卷积核的空间大小，常见的卷积核大小有 3x3、5x5、7x7 等。
# 输入通道数是卷积核需要处理的输入特征图的深度（即输入图像的通道数）。
# 输出通道数是卷积层的过滤器数目，也就是卷积层最终生成的特征图的数量。

# 当你定义一个卷积层时，卷积核的数量会决定卷积层输出的通道数，但卷积核的深度（即每个卷积核的输入通道数）是由输入数据的通道数（即输入特征图的深度）来决定的，而不用手动设置。
# Conv2D 添加一个卷积层，filters=128 表示该层将有 128 个卷积核（即输出通道数），每个卷积核对应一个输出通道。
# kernel_size=(3,3) 表示卷积核的大小是 3x3。trides=1 表示步幅为 1，即卷积核每次移动 1 个像素。padding="same" 表示零填充（zero padding）策略，目的是使得 卷积操作后 输出特征图的尺寸 保持与输入相同（即宽度和高度保持不变）。activation='relu' 使用 ReLU 激活函数。
# 第一层卷积核的形状是 (3, 3, 11，128)，该卷积层的输出图像的形状为(128, 5, 5)，其中 128 是输出的通道数，由于设置了padding = "same"，故输出图像大小不变。
modelCnn.add(Conv2D(filters=128, kernel_size= (3,3), strides=  1 , padding = "same", activation='relu'))

# 另一个卷积层，filters=256 代表使用 256 个卷积核（即输出通道数），每个卷积核对应一个输出通道。padding="valid" 表示不使用填充，输出大小会减少。
# 第二层卷积核的形状是 (3, 3, 128，256)，该卷积层的输出图像的形状为(256, 3, 3)，其中 256 是输出的通道数，由于使用了 valid padding，输出尺寸减少。
modelCnn.add(Conv2D(filters=256, kernel_size= (3,3), strides=  1 , padding = "valid", activation='relu'))

# MaxPool2D 添加一个最大池化层，pool_size=(2,2) 表示 2x2 的池化窗口，池化操作的 步长 默认是 2，减少空间维度（降低特征图的尺寸）。
# 用 2x2 的过滤器，以2为步长进行特征值提取，因此该池化操作会将输入特征图的 空间尺寸（宽度和高度） 缩小一半。采用的池化操作是 最大池化。
# 输入尺寸为 (256, 3, 3)，经过池化后，输出的尺寸会变为 (256, 1, 1)，因为池化操作会将 3x3 的特征图缩小为 1x1。
modelCnn.add(MaxPool2D(pool_size = (2,2)))

# Flatten() 层将二维的特征图展平为一维向量，为全连接层做准备。
# Flatten() 会将池化层输出的 (256, 1, 1) 转换为 256 的一维向量。这是因为在全连接层之前，必须将输入转换为一维数据。
modelCnn.add(Flatten())

# 全连接层处理的是一维向量，而不是多维的图像数据。所以全连接层的输入格式不依赖于 channels_first 或 channels_last。
# Dense(512) 添加一个全连接层，包含 512 个神经元，activation='relu' 使用 ReLU 激活函数。
modelCnn.add(Dense(512, activation='relu'))

# 另一个全连接层，包含 128 个神经元。
modelCnn.add(Dense(128, activation='relu'))

# Dropout(0.4) 是一个正则化方法，随机丢弃 40% 的神经元，防止过拟合。
modelCnn.add(Dropout(0.4))

# 最后一层是一个包含 1 个神经元的全连接层，使用线性激活函数（适合回归任务）。
modelCnn.add(Dense(1, activation='linear'))


# summary() 输出模型的概况，显示每一层的参数数量和输出形状。
modelCnn.summary()

In [ ]:
# 使用 compile() 方法指定模型的损失函数、优化器和评估指标。
# 使用平均绝对误差作为损失函数. 使用 Adam 优化器. 使用平均绝对百分比误差作为评估指标。
modelCnn.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error']) #compile cnn


In [ ]:
# 设置日志文件夹路径。
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# TensorBoard 回调用于在训练过程中记录日志，供 TensorBoard 使用。
# histogram_freq=1 表示每个 epoch 都记录权重的直方图
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

# ModelCheckpoint 回调用于在验证集损失最小化时保存最佳模型。
# HDF5 格式的模型文件，请确保扩展名为 .h5 或 .hdf5
os.makedirs("/content/drive/My Drive/CNNmodels", exist_ok=True)
model_save = ModelCheckpoint("/content/drive/My Drive/CNNmodels/best_CNNmodel.keras",
                             save_best_only=True,
                             save_weights_only=False)  # directory for best model

# model_save = ModelCheckpoint("/content/drive/My Drive/CNNmodels/best_CNNmodel.hdf5", save_best_only = True) #save best model


In [ ]:

# 指定训练数据、测试数据、训练轮次（epochs=100）以及回调函数（tensorboard_callback 和 model_save）。
modelCnn.fit(X_train, y_train, epochs = 100, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, model_save]) #train cnn


In [ ]:

bmodel = load_model('/content/drive/My Drive/CNNmodels/best_CNNmodel.keras')

In [ ]:

y_pred_cnn = bmodel.predict(X_test) #predict best model

In [ ]:
# 计算均方误差（MSE）、均方根误差（RMSE）、平均绝对误差（MAE）和平均绝对百分比误差（MAPE）。
# 这些指标用于评估模型在测试集上的性能。
mse_cnn = mse(y_test, y_pred_cnn) #calculate metrics
rmse_cnn = mse_cnn ** (1/2)
mae_cnn = mae(y_test, y_pred_cnn)
mape_cnn = mape(y_test, y_pred_cnn)

print(mape_cnn)
print(mae_cnn)
print(rmse_cnn)

In [ ]:
# 启动 TensorBoard 可视化工具，查看训练过程中的详细信息，如损失曲线、准确率曲线、权重分布等。
%tensorboard --logdir logs

In [ ]:
# ypred_cnn[:] 和 labels[:] 都是对数组的切片操作，表示返回数组中的所有元素，分别是预测值和真实标签值。
# 无论数组的维度是多少，使用 [:] 都是返回数组中的所有元素。这是 NumPy 中的一个常见用法，它用于获取数组的完整内容，不论数组是多维的。(保持其原有的形状。)

# 计算预测结果的均值
# ypred_cnn 的形状应为 (n, 1)，即一个列向量，每一行代表一个样本的预测结果。如果你打印 ypred_cnn[:]，你将得到模型对所有测试样本的预测结果。
mean_cnn = np.mean(y_pred_cnn[:]) #calculate mean
# 计算预测结果的分位数
quantiles_cnn = np.percentile(y_pred_cnn[:], [1, 25, 50, 75, 99]) #calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99

# 计算标签的均值
# labels 是一个形状为 (n,) 的一维数组，存储了对应测试样本的实际标签（真实的森林高度）。
# labels[:] 会返回所有的标签值。
mean_labels = np.mean(labels[:])
# 计算标签的分位数
quantiles_labels = np.percentile(labels[:], [1, 25, 50, 75, 99])

print(mean_cnn)
print(quantiles_cnn)
# 打印预测值中最小的 10 个和最大的 10 个。
print(np.sort(y_pred_cnn.flatten())[:10]) #print the 10 lowest predictions
print(np.sort(y_pred_cnn.flatten())[-10:][::-1]) #print the 10 highest predictions

print(mean_labels)
print(quantiles_labels)
# 打印标签中最小的 10 个和最大的 10 个。
print(np.sort(labels.flatten())[:10])
print(np.sort(labels.flatten())[-10:][::-1])